In [1]:

import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

import matplotlib.pyplot as plt

import pandas as pd

from src.data_loading import load_price_data
from src.metrics import add_daily_returns
from src.sentiment import add_textblob_sentiment, aggregate_daily_sentiment
from src.news_stock_correlation import (
    align_daily_sentiment_and_returns,
    compute_sentiment_return_correlation,
)
